In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog, simpledialog
import csv
import os
import json
from datetime import datetime

import threading
from typing import List, Dict, Optional

class Book:
    def __init__(self, title: str, author: str, genre: str, publication_year: int, isbn: str = "", description: str = ""):
        self.title = title
        self.author = author
        self.genre = genre
        self.publication_year = publication_year
        self.isbn = isbn
        self.description = description
        self.date_added = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.id = self.generate_id()

    def generate_id(self) -> str:
        import hashlib
        unique_string = f"{self.title}{self.author}{self.publication_year}"
        return hashlib.md5(unique_string.encode()).hexdigest()[:8].upper()

    def to_dict(self) -> Dict:
        return {
            'id': self.id,
            'title': self.title,
            'author': self.author,
            'genre': self.genre,
            'publication_year': self.publication_year,
            'isbn': self.isbn,
            'description': self.description,
            'date_added': self.date_added
        }

class Library:
    def __init__(self):
        self.books: List[Book] = []
        self.genres = set()
        self.authors = set()

    def add_book(self, book: Book) -> Optional[str]:
        for b in self.books:
            if b.title.strip().lower() == book.title.strip().lower() and b.author.lower() == book.author.lower():
                return f"'{book.title}' by {book.author} already exists."
        
        self.books.append(book)
        self.genres.add(book.genre)
        self.authors.add(book.author)
        return None

    def remove_book(self, book_id: str) -> str:
        for i, book in enumerate(self.books):
            if book.id == book_id:
                removed_book = self.books.pop(i)
                self.genres = set(book.genre for book in self.books)
                self.authors = set(book.author for book in self.books)
                return f"'{removed_book.title}' removed successfully."
        return "Book not found."

    def search_books(self, query: str, field: str = "all") -> List[Book]:
        query = query.lower()
        results = []
        
        for book in self.books:
            if field == "all":
                searchable = f"{book.title} {book.author} {book.genre} {book.isbn}".lower()
                if query in searchable:
                    results.append(book)
            elif field == "title" and query in book.title.lower():
                results.append(book)
            elif field == "author" and query in book.author.lower():
                results.append(book)
            elif field == "genre" and query in book.genre.lower():
                results.append(book)
            elif field == "isbn" and query in book.isbn.lower():
                results.append(book)
        
        return results

    def get_book_by_id(self, book_id: str) -> Optional[Book]:
        for book in self.books:
            if book.id == book_id:
                return book
        return None

    def update_book(self, book_id: str, **kwargs) -> str:
        for book in self.books:
            if book.id == book_id:
                old_genre = book.genre
                old_author = book.author
                
                for key, value in kwargs.items():
                    if hasattr(book, key):
                        setattr(book, key, value)
                
                # Update sets
                self.genres = set(book.genre for book in self.books)
                self.authors = set(book.author for book in self.books)
                
                return f"Book updated successfully."
        return "Book not found."

    def get_statistics(self) -> Dict:
        if not self.books:
            return {"total_books": 0, "genres": 0, "authors": 0, "newest_book": None, "oldest_book": None}
        
        years = [book.publication_year for book in self.books]
        newest_year = max(years)
        oldest_year = min(years)
        
        newest_book = next(book for book in self.books if book.publication_year == newest_year)
        oldest_book = next(book for book in self.books if book.publication_year == oldest_year)
        
        return {
            "total_books": len(self.books),
            "genres": len(self.genres),
            "authors": len(self.authors),
            "newest_book": f"{newest_book.title} ({newest_year})",
            "oldest_book": f"{oldest_book.title} ({oldest_year})"
        }

    def save_to_csv(self, filename: str = 'books.csv'):
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(["ID", "Title", "Author", "Genre", "Publication Year", "ISBN", "Description", "Date Added"])
            for book in self.books:
                writer.writerow([book.id, book.title, book.author, book.genre, 
                               book.publication_year, book.isbn, book.description, book.date_added])

    def load_from_csv(self, filename: str = 'books.csv'):
        if not os.path.exists(filename):
            return
        
        self.books = []
        self.genres = set()
        self.authors = set()
        
        with open(filename, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            for row in reader:
                if len(row) >= 5:
                    book = Book(
                        title=row[1],
                        author=row[2],
                        genre=row[3],
                        publication_year=int(row[4]),
                        isbn=row[5] if len(row) > 5 else "",
                        description=row[6] if len(row) > 6 else ""
                    )
                    if len(row) > 7:
                        book.date_added = row[7]
                    if len(row) > 0:
                        book.id = row[0]
                    
                    self.books.append(book)
                    self.genres.add(book.genre)
                    self.authors.add(book.author)

class ProfessionalLibraryGUI:
    def __init__(self, root):
        self.root = root
        self.library = Library()
        self.current_sort_column = None
        self.sort_reverse = False
        self.filtered_books = []
        
        self.setup_main_window()
        self.create_styles()
        self.create_menu()
        self.create_main_interface()
        self.load_library_on_startup()
        self.setup_keyboard_shortcuts()
        
    def setup_main_window(self):
        self.root.title("📚 Professional Library Management Suite v2.0")
        self.root.geometry("1400x900")
        self.root.minsize(1000, 600)
        
        # Professional color theme
        self.theme = {
            'bg_primary': '#FFFFFF',
            'bg_secondary': '#F8F9FA',
            'bg_accent': '#E3F2FD',
            'primary': '#1976D2',
            'primary_dark': '#1565C0',
            'secondary': '#424242',
            'success': '#4CAF50',
            'warning': '#FF9800',
            'error': '#F44336',
            'info': '#2196F3',
            'text_primary': '#212121',
            'text_secondary': '#757575',
            'border': '#E0E0E0'
        }
        
        self.root.configure(bg=self.theme['bg_secondary'])
        
    def create_styles(self):
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Enhanced Treeview styling
        self.style.configure("Professional.Treeview",
                           background=self.theme['bg_primary'],
                           foreground=self.theme['text_primary'],
                           rowheight=35,
                           fieldbackground=self.theme['bg_primary'],
                           font=('Segoe UI', 10))
        
        self.style.configure("Professional.Treeview.Heading",
                           background=self.theme['primary'],
                           foreground='white',
                           font=('Segoe UI', 11, 'bold'),
                           relief='flat')
        
        self.style.map("Professional.Treeview",
                      background=[('selected', self.theme['primary'])],
                      foreground=[('selected', 'white')])

    def create_menu(self):
        menubar = tk.Menu(self.root)
        self.root.config(menu=menubar)
        
        # File menu
        file_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="File", menu=file_menu)
        file_menu.add_command(label="New Library", command=self.new_library)
        file_menu.add_command(label="Save Library", command=self.save_library)
        file_menu.add_command(label="Load Library", command=self.load_library)
        file_menu.add_separator()
        file_menu.add_command(label="Import CSV", command=self.import_csv)
        file_menu.add_command(label="Export CSV", command=self.export_csv)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.root.quit)
        
        # Edit menu
        edit_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Edit", menu=edit_menu)
        edit_menu.add_command(label="Add Book", command=self.add_book_dialog)
        edit_menu.add_command(label="Edit Book", command=self.edit_selected_book)
        edit_menu.add_command(label="Delete Book", command=self.delete_selected_book)
        
        # View menu
        view_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="View", menu=view_menu)
        view_menu.add_command(label="Refresh", command=self.refresh_display)
        view_menu.add_command(label="Statistics", command=self.show_statistics)
        
        # Help menu
        help_menu = tk.Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Help", menu=help_menu)
        help_menu.add_command(label="About", command=self.show_about)

    def create_main_interface(self):
        # Main container
        main_container = tk.Frame(self.root, bg=self.theme['bg_secondary'])
        main_container.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Header section
        self.create_header(main_container)
        
        # Toolbar
        self.create_toolbar(main_container)
        
        # Content area with sidebar
        content_frame = tk.Frame(main_container, bg=self.theme['bg_secondary'])
        content_frame.pack(fill='both', expand=True, pady=(10, 0))
        
        # Main content
        self.create_main_content(content_frame)
        
        # Sidebar
        self.create_sidebar(content_frame)
        
        # Status bar
        self.create_status_bar(main_container)

    def create_header(self, parent):
        header_frame = tk.Frame(parent, bg=self.theme['primary'], height=80)
        header_frame.pack(fill='x')
        header_frame.pack_propagate(False)
        
        # Title
        title_label = tk.Label(header_frame, 
                              text="📚 Professional Library Management Suite",
                              font=('Segoe UI', 20, 'bold'),
                              bg=self.theme['primary'],
                              fg='white')
        title_label.pack(pady=(20, 5))
        
        # Subtitle
        subtitle_label = tk.Label(header_frame,
                                 text="Advanced book collection management system",
                                 font=('Segoe UI', 11),
                                 bg=self.theme['primary'],
                                 fg='white')
        subtitle_label.pack()

    def create_toolbar(self, parent):
        toolbar = tk.Frame(parent, bg=self.theme['bg_primary'], relief='solid', bd=1)
        toolbar.pack(fill='x', pady=(10, 0))
        
        # Button frame
        btn_frame = tk.Frame(toolbar, bg=self.theme['bg_primary'])
        btn_frame.pack(padx=20, pady=15)
        
        # Action buttons with professional styling
        buttons = [
            ("➕ Add Book", self.add_book_dialog, self.theme['success']),
            ("✏️ Edit Book", self.edit_selected_book, self.theme['warning']),
            ("🗑️ Delete Book", self.delete_selected_book, self.theme['error']),
            ("🔍 Search", self.focus_search, self.theme['info']),
            ("📊 Statistics", self.show_statistics, self.theme['secondary']),
            ("💾 Save", self.save_library, self.theme['primary'])
        ]
        
        for i, (text, command, color) in enumerate(buttons):
            btn = tk.Button(btn_frame, text=text, command=command,
                           font=('Segoe UI', 10, 'bold'), 
                           bg=color, fg='white',
                           relief='flat', padx=20, pady=10,
                           cursor='hand2', border=0)
            btn.grid(row=0, column=i, padx=5)
            
            # Hover effect
            self.add_hover_effect(btn, color)

    def create_main_content(self, parent):
        # Left panel for main content
        left_panel = tk.Frame(parent, bg=self.theme['bg_secondary'])
        left_panel.pack(side='left', fill='both', expand=True)
        
        # Search section
        self.create_search_section(left_panel)
        
        # Books table
        self.create_books_table(left_panel)

    def create_search_section(self, parent):
        search_frame = tk.Frame(parent, bg=self.theme['bg_primary'], relief='solid', bd=1)
        search_frame.pack(fill='x', pady=(0, 10))
        
        # Search controls
        controls_frame = tk.Frame(search_frame, bg=self.theme['bg_primary'])
        controls_frame.pack(fill='x', padx=20, pady=15)
        
        # Search label and entry
        tk.Label(controls_frame, text="🔍 Search:", 
                font=('Segoe UI', 11, 'bold'),
                bg=self.theme['bg_primary']).pack(side='left')
        
        self.search_var = tk.StringVar()
        self.search_var.trace('w', self.on_search_change)
        search_entry = tk.Entry(controls_frame, textvariable=self.search_var, 
                               font=('Segoe UI', 11), width=25, relief='solid', bd=1)
        search_entry.pack(side='left', padx=(10, 20))
        
        # Search field selector
        tk.Label(controls_frame, text="in:", 
                font=('Segoe UI', 10),
                bg=self.theme['bg_primary']).pack(side='left')
        
        self.search_field = ttk.Combobox(controls_frame, 
                                        values=['All', 'Title', 'Author', 'Genre', 'ISBN'], 
                                        state='readonly', width=12, font=('Segoe UI', 10))
        self.search_field.set('All')
        self.search_field.pack(side='left', padx=(5, 20))
        self.search_field.bind('<<ComboboxSelected>>', self.on_search_change)
        
        # Genre filter
        tk.Label(controls_frame, text="Genre:", 
                font=('Segoe UI', 10, 'bold'),
                bg=self.theme['bg_primary']).pack(side='left')
        
        self.genre_filter = ttk.Combobox(controls_frame, values=['All Genres'], 
                                        state='readonly', width=15, font=('Segoe UI', 10))
        self.genre_filter.set('All Genres')
        self.genre_filter.pack(side='left', padx=(5, 20))
        self.genre_filter.bind('<<ComboboxSelected>>', self.apply_filters)
        
        # Clear button
        clear_btn = tk.Button(controls_frame, text="Clear", command=self.clear_filters,
                             font=('Segoe UI', 9), bg=self.theme['secondary'], fg='white',
                             relief='flat', padx=15, cursor='hand2')
        clear_btn.pack(side='left')

    def create_books_table(self, parent):
        table_frame = tk.Frame(parent, bg=self.theme['bg_primary'], relief='solid', bd=1)
        table_frame.pack(fill='both', expand=True)
        
        # Table header
        header = tk.Frame(table_frame, bg=self.theme['bg_primary'])
        header.pack(fill='x', padx=20, pady=(15, 10))
        
        tk.Label(header, text="📖 Book Collection", 
                font=('Segoe UI', 14, 'bold'),
                bg=self.theme['bg_primary']).pack(side='left')
        
        self.book_count_label = tk.Label(header, text="0 books",
                                        font=('Segoe UI', 10),
                                        bg=self.theme['bg_primary'],
                                        fg=self.theme['text_secondary'])
        self.book_count_label.pack(side='right')
        
        # Treeview container
        tree_container = tk.Frame(table_frame, bg=self.theme['bg_primary'])
        tree_container.pack(fill='both', expand=True, padx=20, pady=(0, 20))
        
        # Treeview
        columns = ("ID", "Title", "Author", "Genre", "Year", "ISBN", "Added")
        self.tree = ttk.Treeview(tree_container, columns=columns, show='headings', 
                                style='Professional.Treeview', height=18)
        
        # Configure columns
        column_widths = {
            "ID": 80, "Title": 250, "Author": 180, "Genre": 120, 
            "Year": 80, "ISBN": 120, "Added": 100
        }
        
        for col in columns:
            self.tree.heading(col, text=col, command=lambda c=col: self.sort_by_column(c))
            self.tree.column(col, width=column_widths[col], 
                           anchor='center' if col in ['ID', 'Year', 'Added'] else 'w')
        
        # Scrollbars
        v_scrollbar = ttk.Scrollbar(tree_container, orient="vertical", command=self.tree.yview)
        h_scrollbar = ttk.Scrollbar(tree_container, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)
        
        # Layout
        self.tree.grid(row=0, column=0, sticky='nsew')
        v_scrollbar.grid(row=0, column=1, sticky='ns')
        h_scrollbar.grid(row=1, column=0, sticky='ew')
        
        tree_container.grid_rowconfigure(0, weight=1)
        tree_container.grid_columnconfigure(0, weight=1)
        
        # Context menu
        self.create_context_menu()
        
        # Double-click to edit
        self.tree.bind('<Double-1>', lambda e: self.edit_selected_book())

    def create_sidebar(self, parent):
        sidebar = tk.Frame(parent, bg=self.theme['bg_primary'], width=280, relief='solid', bd=1)
        sidebar.pack(side='right', fill='y', padx=(10, 0))
        sidebar.pack_propagate(False)
        
        # Sidebar title
        title_frame = tk.Frame(sidebar, bg=self.theme['primary'])
        title_frame.pack(fill='x')
        tk.Label(title_frame, text="📊 Dashboard", 
                font=('Segoe UI', 12, 'bold'),
                bg=self.theme['primary'], fg='white', pady=10).pack()
        
        # Quick stats
        stats_frame = tk.LabelFrame(sidebar, text="Quick Statistics", 
                                   font=('Segoe UI', 10, 'bold'), padx=15, pady=15)
        stats_frame.pack(fill='x', padx=15, pady=15)
        
        self.stats_labels = {}
        stats = [("Total Books", "📚"), ("Authors", "✍️"), ("Genres", "📂"), ("This Month", "📅")]
        
        for i, (stat, icon) in enumerate(stats):
            frame = tk.Frame(stats_frame)
            frame.pack(fill='x', pady=5)
            
            tk.Label(frame, text=f"{icon} {stat}:", 
                    font=('Segoe UI', 9)).pack(side='left')
            
            label = tk.Label(frame, text="0", font=('Segoe UI', 9, 'bold'))
            label.pack(side='right')
            self.stats_labels[stat] = label
        
        # Recent activity
        activity_frame = tk.LabelFrame(sidebar, text="Recent Activity", 
                                      font=('Segoe UI', 10, 'bold'), padx=15, pady=15)
        activity_frame.pack(fill='both', expand=True, padx=15, pady=(0, 15))
        
        # Activity listbox with scrollbar
        activity_container = tk.Frame(activity_frame)
        activity_container.pack(fill='both', expand=True)
        
        self.activity_list = tk.Listbox(activity_container, font=('Segoe UI', 8),
                                       selectmode='none', height=10)
        activity_scroll = ttk.Scrollbar(activity_container, orient="vertical", 
                                       command=self.activity_list.yview)
        self.activity_list.configure(yscrollcommand=activity_scroll.set)
        
        self.activity_list.pack(side='left', fill='both', expand=True)
        activity_scroll.pack(side='right', fill='y')

    def create_status_bar(self, parent):
        status_frame = tk.Frame(parent, bg=self.theme['secondary'], height=25)
        status_frame.pack(fill='x', side='bottom', pady=(10, 0))
        status_frame.pack_propagate(False)
        
        self.status_var = tk.StringVar(value="Ready")
        status_label = tk.Label(status_frame, textvariable=self.status_var,
                               font=('Segoe UI', 9), bg=self.theme['secondary'], 
                               fg='white', anchor='w')
        status_label.pack(side='left', fill='x', expand=True, padx=10, pady=3)
        
        # Time display
        self.time_var = tk.StringVar()
        time_label = tk.Label(status_frame, textvariable=self.time_var,
                             font=('Segoe UI', 9), bg=self.theme['secondary'], 
                             fg='white')
        time_label.pack(side='right', padx=10, pady=3)
        
        self.update_time()

    def create_context_menu(self):
        self.context_menu = tk.Menu(self.root, tearoff=0)
        self.context_menu.add_command(label="📖 View Details", command=self.view_book_details)
        self.context_menu.add_command(label="✏️ Edit Book", command=self.edit_selected_book)
        self.context_menu.add_command(label="📋 Copy Title", command=self.copy_book_title)
        self.context_menu.add_separator()
        self.context_menu.add_command(label="🗑️ Delete Book", command=self.delete_selected_book)
        
        self.tree.bind("<Button-3>", self.show_context_menu)

    # Event handlers and utility methods
    def add_hover_effect(self, button, color):
        original_color = color
        hover_color = self.darken_color(color)
        
        def on_enter(e):
            button.configure(bg=hover_color)
        def on_leave(e):
            button.configure(bg=original_color)
            
        button.bind("<Enter>", on_enter)
        button.bind("<Leave>", on_leave)

    def darken_color(self, color):
        color_map = {
            self.theme['success']: '#45A049',
            self.theme['warning']: '#E68900',
            self.theme['error']: '#D32F2F',
            self.theme['info']: '#1976D2',
            self.theme['primary']: '#1565C0',
            self.theme['secondary']: '#303030'
        }
        return color_map.get(color, color)

    def update_time(self):
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.time_var.set(current_time)
        self.root.after(1000, self.update_time)

    def update_status(self, message, timeout=3000):
        self.status_var.set(message)
        if timeout > 0:
            self.root.after(timeout, lambda: self.status_var.set("Ready"))

    def log_activity(self, activity):
        timestamp = datetime.now().strftime("%H:%M")
        self.activity_list.insert(0, f"{timestamp} - {activity}")
        if self.activity_list.size() > 20:
            self.activity_list.delete(20, 'end')

    def update_stats(self):
        stats = self.library.get_statistics()
        self.stats_labels["Total Books"].configure(text=str(stats["total_books"]))
        self.stats_labels["Authors"].configure(text=str(stats["authors"]))
        self.stats_labels["Genres"].configure(text=str(stats["genres"]))
        
        # Books added this month
        current_month = datetime.now().strftime("%Y-%m")
        this_month = sum(1 for book in self.library.books 
                        if book.date_added.startswith(current_month))
        self.stats_labels["This Month"].configure(text=str(this_month))

    def update_genre_filter(self):
        genres = ['All Genres'] + sorted(list(self.library.genres))
        self.genre_filter['values'] = genres

    def update_book_count(self):
        total = len(self.library.books)
        shown = len(self.filtered_books)
        if total == shown:
            text = f"{total} books"
        else:
            text = f"{shown} of {total} books"
        self.book_count_label.configure(text=text)

    # Core functionality
    def on_search_change(self, *args):
        self.apply_filters()

    def apply_filters(self, event=None):
        search_query = self.search_var.get().strip()
        search_field = self.search_field.get().lower()
        genre_filter = self.genre_filter.get()
        
        # Start with all books
        books = self.library.books[:]
        
        # Apply search filter
        if search_query:
            if search_field == 'all':
                books = self.library.search_books(search_query, "all")
            else:
                books = self.library.search_books(search_query, search_field)
        
        # Apply genre filter
        if genre_filter and genre_filter != 'All Genres':
            books = [book for book in books if book.genre == genre_filter]
        
        self.filtered_books = books
        self.display_books()

    def clear_filters(self):
        self.search_var.set("")
        self.search_field.set("All")
        self.genre_filter.set("All Genres")
        self.apply_filters()

    def sort_by_column(self, column):
        if self.current_sort_column == column:
            self.sort_reverse = not self.sort_reverse
        else:
            self.current_sort_column = column
            self.sort_reverse = False
        self.display_books()

    def display_books(self):
        # Clear existing items
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Sort books
        books = self.filtered_books[:]
        if self.current_sort_column:
            reverse = self.sort_reverse
            if self.current_sort_column == "Title":
                books.sort(key=lambda b: b.title.lower(), reverse=reverse)
            elif self.current_sort_column == "Author":
                books.sort(key=lambda b: b.author.lower(), reverse=reverse)
            elif self.current_sort_column == "Genre":
                books.sort(key=lambda b: b.genre.lower(), reverse=reverse)
            elif self.current_sort_column == "Year":
                books.sort(key=lambda b: b.publication_year, reverse=reverse)
            elif self.current_sort_column == "Added":
                books.sort(key=lambda b: b.date_added, reverse=reverse)
        
        # Insert books
        for i, book in enumerate(books):
            tags = ('evenrow',) if i % 2 == 0 else ('oddrow',)
            date_short = book.date_added.split()[0] if ' ' in book.date_added else book.date_added
            
            self.tree.insert('', 'end',
                           values=(book.id, book.title, book.author, book.genre,
                                  book.publication_year, book.isbn, date_short),
                           tags=tags)
        
        # Configure alternating row colors
        self.tree.tag_configure('evenrow', background='#F8F9FA')
        self.tree.tag_configure('oddrow', background=self.theme['bg_primary'])
        
        self.update_book_count()

    def refresh_display(self):
        self.apply_filters()
        self.update_stats()
        self.update_genre_filter()
        self.update_status("Display refreshed")

    # Dialog methods
    def create_book_dialog(self, title, book=None):
        dialog = tk.Toplevel(self.root)
        dialog.title(title)
        dialog.geometry("500x400")
        dialog.configure(bg=self.theme['bg_primary'])
        dialog.resizable(False, False)
        dialog.transient(self.root)
        dialog.grab_set()
        
        # Center dialog
        dialog.geometry("+{}+{}".format(
            int(self.root.winfo_x() + self.root.winfo_width()/2 - 250),
            int(self.root.winfo_y() + self.root.winfo_height()/2 - 200)
        ))
        
        # Dialog header
        header = tk.Frame(dialog, bg=self.theme['primary'])
        header.pack(fill='x')
        tk.Label(header, text=title, font=('Segoe UI', 16, 'bold'),
                bg=self.theme['primary'], fg='white', pady=15).pack()
        
        # Form frame
        form_frame = tk.Frame(dialog, bg=self.theme['bg_primary'])
        form_frame.pack(fill='both', expand=True, padx=30, pady=20)
        
        # Form fields
        fields = {}
        field_names = [
            ('Title', 'title'),
            ('Author', 'author'),
            ('Genre', 'genre'),
            ('Publication Year', 'publication_year'),
            ('ISBN (Optional)', 'isbn'),
            ('Description (Optional)', 'description')
        ]
        
        for i, (label_text, field_name) in enumerate(field_names):
            # Label
            tk.Label(form_frame, text=label_text + ":",
                    font=('Segoe UI', 10, 'bold'),
                    bg=self.theme['bg_primary']).grid(row=i, column=0, sticky='w', pady=(0, 5))
            
            # Entry or Text widget
            if field_name == 'description':
                widget = tk.Text(form_frame, font=('Segoe UI', 10), height=3, width=40,
                               relief='solid', bd=1)
                if book:
                    widget.insert('1.0', getattr(book, field_name, ''))
            else:
                widget = tk.Entry(form_frame, font=('Segoe UI', 10), width=40,
                                relief='solid', bd=1)
                if book:
                    widget.insert(0, str(getattr(book, field_name, '')))
            
            widget.grid(row=i, column=1, pady=(0, 15), sticky='w')
            fields[field_name] = widget
        
        # Focus first field
        fields['title'].focus()
        
        # Button frame
        button_frame = tk.Frame(dialog, bg=self.theme['bg_primary'])
        button_frame.pack(fill='x', padx=30, pady=(0, 20))
        
        result = {'success': False, 'data': {}}
        
        def on_save():
            try:
                # Get values
                data = {}
                for field_name, widget in fields.items():
                    if field_name == 'description':
                        value = widget.get('1.0', 'end-1c').strip()
                    else:
                        value = widget.get().strip()
                    data[field_name] = value
                
                # Validate required fields
                required_fields = ['title', 'author', 'genre', 'publication_year']
                for field in required_fields:
                    if not data[field]:
                        messagebox.showerror("Error", f"{field.replace('_', ' ').title()} is required!")
                        return
                
                # Validate year
                try:
                    data['publication_year'] = int(data['publication_year'])
                except ValueError:
                    messagebox.showerror("Error", "Publication year must be a valid number!")
                    return
                
                result['success'] = True
                result['data'] = data
                dialog.destroy()
                
            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {str(e)}")
        
        def on_cancel():
            dialog.destroy()
        
        # Buttons
        tk.Button(button_frame, text="Save", command=on_save,
                 font=('Segoe UI', 11, 'bold'), bg=self.theme['success'], fg='white',
                 relief='flat', padx=30, pady=8, cursor='hand2').pack(side='right', padx=(10, 0))
        
        tk.Button(button_frame, text="Cancel", command=on_cancel,
                 font=('Segoe UI', 11, 'bold'), bg=self.theme['error'], fg='white',
                 relief='flat', padx=30, pady=8, cursor='hand2').pack(side='right')
        
        dialog.wait_window()
        return result

    def add_book_dialog(self):
        result = self.create_book_dialog("Add New Book")
        if result['success']:
            data = result['data']
            book = Book(
                title=data['title'],
                author=data['author'],
                genre=data['genre'],
                publication_year=data['publication_year'],
                isbn=data['isbn'],
                description=data['description']
            )
            
            error = self.library.add_book(book)
            if error:
                messagebox.showwarning("Duplicate Book", error)
            else:
                self.refresh_display()
                self.log_activity(f"Added book: {book.title}")
                self.update_status(f"Book '{book.title}' added successfully")
                messagebox.showinfo("Success", f"Book '{book.title}' added successfully!")

    def edit_selected_book(self):
        selection = self.tree.selection()
        if not selection:
            messagebox.showwarning("No Selection", "Please select a book to edit.")
            return
        
        item = self.tree.item(selection[0])
        book_id = item['values'][0]
        book = self.library.get_book_by_id(book_id)
        
        if not book:
            messagebox.showerror("Error", "Book not found!")
            return
        
        result = self.create_book_dialog("Edit Book", book)
        if result['success']:
            data = result['data']
            self.library.update_book(
                book_id,
                title=data['title'],
                author=data['author'],
                genre=data['genre'],
                publication_year=data['publication_year'],
                isbn=data['isbn'],
                description=data['description']
            )
            
            self.refresh_display()
            self.log_activity(f"Updated book: {data['title']}")
            self.update_status(f"Book '{data['title']}' updated successfully")
            messagebox.showinfo("Success", "Book updated successfully!")

    def delete_selected_book(self):
        selection = self.tree.selection()
        if not selection:
            messagebox.showwarning("No Selection", "Please select a book to delete.")
            return
        
        item = self.tree.item(selection[0])
        book_id = item['values'][0]
        book_title = item['values'][1]
        
        if messagebox.askyesno("Confirm Delete", 
                              f"Are you sure you want to delete '{book_title}'?",
                              icon='warning'):
            result = self.library.remove_book(book_id)
            self.refresh_display()
            self.log_activity(f"Deleted book: {book_title}")
            self.update_status(result)

    def view_book_details(self):
        selection = self.tree.selection()
        if not selection:
            messagebox.showwarning("No Selection", "Please select a book to view details.")
            return
        
        item = self.tree.item(selection[0])
        book_id = item['values'][0]
        book = self.library.get_book_by_id(book_id)
        
        if not book:
            messagebox.showerror("Error", "Book not found!")
            return
        
        # Create details dialog
        details_window = tk.Toplevel(self.root)
        details_window.title(f"Book Details - {book.title}")
        details_window.geometry("500x400")
        details_window.configure(bg=self.theme['bg_primary'])
        details_window.resizable(False, False)
        details_window.transient(self.root)
        
        # Header
        header = tk.Frame(details_window, bg=self.theme['primary'])
        header.pack(fill='x')
        tk.Label(header, text="📖 Book Details", font=('Segoe UI', 16, 'bold'),
                bg=self.theme['primary'], fg='white', pady=15).pack()
        
        # Details frame
        details_frame = tk.Frame(details_window, bg=self.theme['bg_primary'])
        details_frame.pack(fill='both', expand=True, padx=30, pady=20)
        
        # Book information
        info_items = [
            ("ID:", book.id),
            ("Title:", book.title),
            ("Author:", book.author),
            ("Genre:", book.genre),
            ("Publication Year:", str(book.publication_year)),
            ("ISBN:", book.isbn or "N/A"),
            ("Date Added:", book.date_added),
            ("Description:", book.description or "No description available")
        ]
        
        for i, (label, value) in enumerate(info_items):
            tk.Label(details_frame, text=label, font=('Segoe UI', 10, 'bold'),
                    bg=self.theme['bg_primary'], anchor='w').grid(row=i, column=0, sticky='nw', pady=5)
            
            if label == "Description:":
                text_widget = tk.Text(details_frame, font=('Segoe UI', 10), height=6, width=40,
                                    wrap='word', relief='solid', bd=1, state='normal')
                text_widget.insert('1.0', value)
                text_widget.configure(state='disabled')
                text_widget.grid(row=i, column=1, sticky='w', pady=5, padx=(10, 0))
            else:
                tk.Label(details_frame, text=value, font=('Segoe UI', 10),
                        bg=self.theme['bg_primary'], anchor='w', wraplength=300).grid(row=i, column=1, sticky='w', pady=5, padx=(10, 0))

    def copy_book_title(self):
        selection = self.tree.selection()
        if not selection:
            return
        
        item = self.tree.item(selection[0])
        title = item['values'][1]
        self.root.clipboard_clear()
        self.root.clipboard_append(title)
        self.update_status(f"Copied '{title}' to clipboard")

    def show_context_menu(self, event):
        try:
            self.context_menu.tk_popup(event.x_root, event.y_root)
        finally:
            self.context_menu.grab_release()

    def focus_search(self):
        # Focus on search entry
        for child in self.root.winfo_children():
            for widget in child.winfo_children():
                if hasattr(widget, 'winfo_children'):
                    for subwidget in widget.winfo_children():
                        if isinstance(subwidget, tk.Entry) and str(subwidget.cget('textvariable')) == str(self.search_var):
                            subwidget.focus()
                            return

    # File operations
    def new_library(self):
        if len(self.library.books) > 0:
            if not messagebox.askyesno("New Library", "This will clear all current books. Continue?"):
                return
        
        self.library = Library()
        self.refresh_display()
        self.log_activity("Created new library")
        self.update_status("New library created")

    def save_library(self):
        try:
            self.library.save_to_csv('books.csv')
            self.log_activity("Library saved")
            self.update_status("Library saved successfully")
            messagebox.showinfo("Success", "Library saved successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save library: {str(e)}")

    def load_library(self):
        try:
            if len(self.library.books) > 0:
                if not messagebox.askyesno("Load Library", "This will replace current books. Continue?"):
                    return
            
            self.library.load_from_csv('books.csv')
            self.refresh_display()
            self.log_activity("Library loaded")
            self.update_status("Library loaded successfully")
            messagebox.showinfo("Success", "Library loaded successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load library: {str(e)}")

    def import_csv(self):
        filename = filedialog.askopenfilename(
            title="Import CSV File",
            filetypes=[("CSV files", "*.csv"), ("All files", "*.*")]
        )
        if filename:
            try:
                self.library.load_from_csv(filename)
                self.refresh_display()
                self.log_activity(f"Imported from {os.path.basename(filename)}")
                self.update_status("CSV imported successfully")
                messagebox.showinfo("Success", "CSV file imported successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to import CSV: {str(e)}")

    def export_csv(self):
        filename = filedialog.asksaveasfilename(
            title="Export CSV File",
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("All files", "*.*")]
        )
        if filename:
            try:
                self.library.save_to_csv(filename)
                self.log_activity(f"Exported to {os.path.basename(filename)}")
                self.update_status("CSV exported successfully")
                messagebox.showinfo("Success", "CSV file exported successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to export CSV: {str(e)}")

    def show_statistics(self):
        stats = self.library.get_statistics()
        
        stats_window = tk.Toplevel(self.root)
        stats_window.title("📊 Library Statistics")
        stats_window.geometry("400x300")
        stats_window.configure(bg=self.theme['bg_primary'])
        stats_window.resizable(False, False)
        stats_window.transient(self.root)
        
        # Header
        header = tk.Frame(stats_window, bg=self.theme['primary'])
        header.pack(fill='x')
        tk.Label(header, text="📊 Library Statistics", font=('Segoe UI', 16, 'bold'),
                bg=self.theme['primary'], fg='white', pady=15).pack()
        
        # Stats frame
        stats_frame = tk.Frame(stats_window, bg=self.theme['bg_primary'])
        stats_frame.pack(fill='both', expand=True, padx=30, pady=20)
        
        # Statistics display
        if stats['total_books'] > 0:
            stat_items = [
                ("📚 Total Books:", str(stats['total_books'])),
                ("✍️ Number of Authors:", str(stats['authors'])),
                ("📂 Number of Genres:", str(stats['genres'])),
                ("🆕 Newest Book:", stats['newest_book']),
                ("📜 Oldest Book:", stats['oldest_book'])
            ]
        else:
            stat_items = [("📚 Total Books:", "0"), ("Status:", "Library is empty")]
        
        for i, (label, value) in enumerate(stat_items):
            tk.Label(stats_frame, text=label, font=('Segoe UI', 11, 'bold'),
                    bg=self.theme['bg_primary'], anchor='w').grid(row=i, column=0, sticky='w', pady=8)
            tk.Label(stats_frame, text=value, font=('Segoe UI', 11),
                    bg=self.theme['bg_primary'], anchor='w').grid(row=i, column=1, sticky='w', pady=8, padx=(20, 0))

    def show_about(self):
        about_text = """
Professional Library Management Suite v2.0

A comprehensive book collection management system 
with advanced features for organizing, searching, 
and managing your personal library.

Features:
• Advanced search and filtering
• Professional user interface
• Import/Export capabilities
• Real-time statistics
• Activity logging
• Keyboard shortcuts

Developed with Python and Tkinter
        """
        messagebox.showinfo("About", about_text)

    def show_shortcuts(self):
        shortcuts_text = """
Keyboard Shortcuts:

Ctrl+N    - New Library
Ctrl+S    - Save Library  
Ctrl+A    - Add Book
Ctrl+E    - Edit Selected Book
Ctrl+F    - Focus Search
Ctrl+I    - Show Statistics
F5        - Refresh Display
Delete    - Delete Selected Book

Mouse Actions:
Double-click - Edit Book
Right-click  - Context Menu
        """
        messagebox.showinfo("Keyboard Shortcuts", shortcuts_text)

    def setup_keyboard_shortcuts(self):
        self.root.bind('<Control-n>', lambda e: self.new_library())
        self.root.bind('<Control-s>', lambda e: self.save_library())
        self.root.bind('<Control-a>', lambda e: self.add_book_dialog())
        self.root.bind('<Control-e>', lambda e: self.edit_selected_book())
        self.root.bind('<Control-f>', lambda e: self.focus_search())
        self.root.bind('<Control-i>', lambda e: self.show_statistics())
        self.root.bind('<F5>', lambda e: self.refresh_display())
        self.root.bind('<Delete>', lambda e: self.delete_selected_book())

    def load_library_on_startup(self):
        try:
            if os.path.exists('books.csv'):
                self.library.load_from_csv('books.csv')
                self.refresh_display()
                self.log_activity("Library loaded on startup")
                self.update_status("Library loaded from books.csv")
        except Exception as e:
            self.log_activity("Failed to load library on startup")
            self.update_status("No existing library found")

# Main Application
if __name__ == "__main__":
    root = tk.Tk()
    app = ProfessionalLibraryGUI(root)
    root.mainloop()